In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [0]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
from PIL import Image
import dlib
from skimage import io
import matplotlib.pyplot as plt


In [0]:
! git clone https://github.com/Skuldur/facenet-face-recognition.git

In [0]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [3]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))    
    return loss

In [0]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

In [0]:
load_weights_from_FaceNet(FRmodel)

In [0]:
def save_to_database(image_name):
    image = io.imread(image_name)
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(), x.right(), x.bottom()) for x in detected_faces]
    if len(face_frames) != 0:
        face_rect = face_frames[0]
        face = Image.fromarray(image).crop(face_rect)
        img = face.resize((96,96), Image.ANTIALIAS)
        img.save(image_name)
    else:
        img = Image.open(image_name)
        img = img.resize((96,96), Image.ANTIALIAS)
        img.save(image_name)

In [0]:
def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    
    distances = []
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding - db_enc)
        distances.append([dist, name])
        if dist < min_dist:
            min_dist = dist
            identity = name
    distances.sort()
#     if min_dist > 0.80:
#         print("Not in the database.")
#     else:
    print("Here are top 3 guesses:")
    for i in range(4):
        print("Guess " + str(i+1) + ": " + distances[i][1] + " (Confidence: " + str(round((1-distances[i][0])* 100,2)) + "%)")
        #print ("it's " + str(identity) + ", the distance is " + str(min_dist))
#     print(distances)    
#     return min_dist, identity                                                                                  

In [0]:
def test_image(image_name):
    image = io.imread(image_name)
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(), x.right(), x.bottom()) for x in detected_faces]
    if len(face_frames) != 0:
        face_rect = face_frames[0]
        face = Image.fromarray(image).crop(face_rect)
        img = face.resize((96,96), Image.ANTIALIAS)
        img.save(image_name)
    else:
        img = Image.open(image_name)
        img = img.resize((96,96), Image.ANTIALIAS)
        img.save(image_name)
#     imgplot = plt.imshow(img)
#     plt.show()

In [0]:
user_ids = ['dhoni','kohli','dravid','sachin', 'user1', 'user2']
database = {}

In [0]:
for id in user_ids:
    filename = './' + str(id) + '.jpg'
    save_to_database(filename)
    database[str(id)] = img_to_encoding(filename, FRmodel)

In [0]:
def identify(filename):
    test_image(filename)
    who_is_it(filename, database, FRmodel)

In [165]:
identify('./test2.jpg')

Here are top 3 guesses:
Guess 1: user2 (Confidence: 10.56%)
Guess 2: sachin (Confidence: -7.19%)
Guess 3: dravid (Confidence: -8.29%)
Guess 4: kohli (Confidence: -25.31%)
